In [0]:

# path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_category.v1/fact/granularity=daily/month=2018-02/device_id=2001/store_id=143445/"
# spark.read.parquet(path).filter("app_id=100 and date='2018-02-28'").show()


# path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.basic-kpi.v2/fact/granularity=monthly/date=2015-12-31/"
# spark.read.parquet(path).printSchema()
# collect_list = spark.read.parquet(path).filter("device_code='android-phone' and app_id='20600000272436' and country_code='US'").collect()
# print collect_list
# print len(collect_list[0])
# for record in collect_list[0]:
#     print record

raw_path_1="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.basic-kpi.v2/fact/granularity=monthly/date=2013-01-31/"
spark.read.parquet(raw_path_1).printSchema()

spark.read.parquet(raw_path_1).filter("app_id='20600000000014' and country_code='AU' and device_code='android-phone'").show()


# path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v1/fact/granularity=monthly/month=2018-02/device_id=1001/store_id=1/"
# spark.read.parquet(path).printSchema()
# spark.read.parquet(path).filter("app_id='20600003808593' and seg_app_id='20600000011090'").orderBy('kpi').show()


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 
-- SELECT date
-- FROM plproxy.execute_select_nestloop(\$proxy\$
--	select date from au.app_monthly_2002_143445_201801
-- \$proxy\$) t (date DATE) limit 3;

-- SELECT device_id,store_id,date,app_id,kpi,estimate
-- FROM plproxy.execute_select_nestloop(\$proxy\$
-- select device_id,store_id,date,app_id,kpi,estimate from mu.app_monthly_2002_0_201601
--\$proxy\$) t (device_id SMALLINT,store_id INT,date DATE,app_id BIGINT,kpi SMALLINT, estimate FLOAT) where app_id in (SELECT app_id
-- FROM plproxy.execute_select_nestloop(\$proxy\$
--select app_id from mu.app_monthly_2002_0_201601
--\$proxy\$) t (app_id BIGINT) limit 1 ) order by kpi  ;

SELECT device_id,store_id,date,app_id,kpi,estimate FROM plproxy.execute_select_nestloop(\$proxy\$
select device_id,store_id,date,app_id,kpi,estimate from mu.app_monthly_2002_143447_201501\$proxy\$) t (device_id SMALLINT,store_id INT,date DATE,app_id BIGINT,kpi SMALLINT, estimate FLOAT) where app_id in (SELECT app_id FROM plproxy.execute_select_nestloop(\$proxy\$select app_id from mu.app_monthly_2002_143447_201501\$proxy\$) t (app_id BIGINT) limit 1 ) order by kpi  ;


-- SELECT device_id,store_id,date,app_id,kpi,estimate,category_id,rank
-- FROM plproxy.execute_select_nestloop(\$proxy\$
-- select device_id,store_id,date,app_id,kpi,estimate, category_id,rank from mu.category_daily_2001_143445_201802
-- \$proxy\$) t (device_id SMALLINT,store_id INT,date DATE,app_id BIGINT,kpi SMALLINT, estimate FLOAT, category_id INT, rank INT) where app_id=100 and date='2018-02-28' and device_id='2001' limit 20;


-- SELECT device_id,store_id,date,app_id,kpi,estimate,seg_app_id
--  FROM plproxy.execute_select_nestloop(\$proxy\$select device_id,store_id,date,app_id,kpi,estimate,seg_app_id from au.app_monthly_1001_1_201802
-- \$proxy\$) t (device_id SMALLINT,store_id INT,date DATE,app_id BIGINT,kpi SMALLINT, estimate FLOAT, seg_app_id BIGINT) where app_id='20600003808593' and seg_app_id='20600000011090' order by kpi asc limit 30;

EOF

143447 2002 2015-01

In [0]:

import unittest
import datetime
from dateutil.relativedelta import relativedelta
from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import count
from pyspark.sql import Row

kpi_mapping={1:"est_average_active_users", 2: "est_average_session_per_user", 3:"est_average_session_duration", 4:"est_install_penetration", 5:"est_average_active_days", 6:"est_percentage_active_days", 7:"est_average_bytes_per_user" , 8:"est_average_time_per_user", 9:"est_usage_penetration", 10:"est_open_rate",11:"est_total_time", 12:"est_share_of_category_time",14:"est_total_sessions", 15:"est_share_of_category_session", 17:"est_average_bytes_per_session", 18:"est_share_of_category_bytes", 20:"est_percent_of_wifi_total", 21:"est_mb_per_second", 22:"est_panel_size", 23:"est_installs", 24:"est_average_active_users_country_share", 25:"est_installs_country_share", 26:"est_audience_index", 27:"est_audience_percentage",28:"est_cross_product_affinity"}

ANDROID_COUNTRY_ID_CODES = { 1: 'AU', 2: 'CA', 3: 'CN', 4: 'DE', 5: 'ES', 6: 'FR', 7: 'GB', 8: 'IT', 9: 'JP', 10: 'US', 11: 'BE', 12: 'CH', 13: 'CL', 14: 'ZA', 15: 'VN', 16: 'HK', 17: 'AR', 18: 'BR', 19: 'IN', 20: 'FI', 21: 'ID', 22: 'RU', 23: 'NL', 24: 'MY', 25: 'TR', 26: 'MX', 27: 'KR', 28: 'PL', 29: 'TH', 30: 'TW', 31: 'PH', 32: 'SG', 33: 'EG', 34: 'SE', 35: 'AT', 36: 'CZ', 37: 'HU', 38: 'DK', 39: 'IE', 40: 'IL', 41: 'NZ', 42: 'NO', 43: 'PT', 44: 'RO', 45: 'SK', 46: 'GR', 47: 'BG', 48: 'UA', 49: 'AE', 50: 'KW', 51: 'SA', 52: 'CO', 65: 'LB', 56: 'PE', 80: 'HR', 54: 'PK', 62: 'EC', 73: 'QA', 102: 'MO', 103: 'LU', 53: 'KZ', 1000: 'WW' }


IOS_COUNTRY_ID_CODES={143460:'AU',143455:'CA',143465:'CN',143443:'DE',143454:'ES',143442:'FR',143444:'GB',143450:'IT',143462:'JP',143441:'US',143446:'BE',143459:'CH',143483:'CL',143472:'ZA',143471:'VN',143463:'HK',143505:'AR',143503:'BR',143467:'IN',143447:'FI',143476:'ID',143469:'RU',143452:'NL',143473:'MY',143480:'TR',143468:'MX',143466:'KR',143478:'PL',143475:'TH',143470:'TW',143474:'PH',143464:'SG',143516:'EG',143456:'SE',143445:'AT',143489:'CZ',143482:'HU',143458:'DK',143449:'IE',143491:'IL',143461:'NZ',143457:'NO',143453:'PT',143487:'RO',143496:'SK',143448:'GR',143526:'BG',143492:'UA',143481:'AE',143493:'KW',143479:'SA',143501:'CO',143451:'LU',143497:'LB',143515:'MO',143507:'PE',143494:'HR',143477:'PK',143509:'EC',143498:'QA',0:'WW'}

def get_device_list(device):
    if device=='ios':
        return {"2001":"ios-phone","2002":"ios-tablet"}
    else:
        return {"1001":"android-phone" ,"1002":"android-tablet"}


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)  
    return next_month - datetime.timedelta(days=next_month.day)


def get_month_list():
    result = []
    today = datetime.date(2019, 10, 1) 
    current = datetime.date(2013, 1, 1)    
    while current <= today:
        month_data_raw=datetime.datetime.strftime(current,'%Y-%m')
        month_data_leg_db=datetime.datetime.strftime(current,'%Y%m')
        result.append((month_data_raw, month_data_leg_db,current  ))
        current += relativedelta(months=1)  
    return result


def get_path_date_list(granularity):
    # df_date = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v1/fact/granularity={}/".format(granularity)).select('date').dropDuplicates()
    # collect_date= df_date.collect()
    collect_date=[Row(date=u'2014-02-22'), Row(date=u'2014-12-13'), Row(date=u'2019-08-31'), Row(date=u'2016-08-20'), Row(date=u'2016-05-21'), Row(date=u'2015-02-28'), Row(date=u'2013-12-28'), Row(date=u'2014-11-01'), Row(date=u'2015-08-15'), Row(date=u'2016-01-16'), Row(date=u'2018-12-22'), Row(date=u'2016-07-23'), Row(date=u'2014-02-08'), Row(date=u'2017-03-18'), Row(date=u'2019-06-08'), Row(date=u'2019-07-13'), Row(date=u'2019-01-26'), Row(date=u'2016-04-09'), Row(date=u'2016-07-30'), Row(date=u'2018-07-21'), Row(date=u'2015-02-07'), Row(date=u'2013-10-05'), Row(date=u'2018-09-15'), Row(date=u'2013-02-09'), Row(date=u'2015-10-10'), Row(date=u'2018-08-04'), Row(date=u'2019-06-15'), Row(date=u'2018-07-28'), Row(date=u'2018-12-15'), Row(date=u'2019-06-22'), Row(date=u'2016-01-30'), Row(date=u'2014-02-15'), Row(date=u'2014-10-18'), Row(date=u'2014-06-28'), Row(date=u'2014-05-24'), Row(date=u'2017-09-23'), Row(date=u'2015-01-10'), Row(date=u'2016-01-23'), Row(date=u'2016-10-15'), Row(date=u'2014-09-27'), Row(date=u'2015-03-28'), Row(date=u'2013-06-15'), Row(date=u'2018-06-09'), Row(date=u'2018-04-28'), Row(date=u'2019-01-05'), Row(date=u'2017-06-24'), Row(date=u'2014-09-20'), Row(date=u'2017-12-02'), Row(date=u'2013-04-06'), Row(date=u'2013-08-03'), Row(date=u'2014-04-05'), Row(date=u'2013-10-26'), Row(date=u'2019-11-16'), Row(date=u'2016-10-08'), Row(date=u'2019-01-12'), Row(date=u'2018-09-29'), Row(date=u'2017-07-29'), Row(date=u'2019-04-06'), Row(date=u'2018-02-17'), Row(date=u'2019-10-12'), Row(date=u'2016-10-01'), Row(date=u'2014-05-10'), Row(date=u'2018-06-30'), Row(date=u'2018-12-01'), Row(date=u'2013-09-28'), Row(date=u'2017-06-03'), Row(date=u'2013-07-27'), Row(date=u'2016-11-19'), Row(date=u'2013-11-16'), Row(date=u'2015-05-02'), Row(date=u'2017-02-11'), Row(date=u'2013-07-06'), Row(date=u'2015-01-24'), Row(date=u'2013-03-23'), Row(date=u'2019-09-21'), Row(date=u'2016-12-31'), Row(date=u'2018-04-14'), Row(date=u'2018-07-07'), Row(date=u'2018-08-18'), Row(date=u'2019-10-05'), Row(date=u'2018-11-17'), Row(date=u'2017-02-04'), Row(date=u'2017-12-09'), Row(date=u'2015-11-28'), Row(date=u'2014-04-12'), Row(date=u'2013-12-14'), Row(date=u'2017-08-26'), Row(date=u'2018-06-23'), Row(date=u'2018-01-20'), Row(date=u'2016-05-14'), Row(date=u'2018-10-13'), Row(date=u'2015-02-14'), Row(date=u'2015-07-11'), Row(date=u'2015-07-18'), Row(date=u'2014-07-26'), Row(date=u'2014-11-08'), Row(date=u'2015-05-30'), Row(date=u'2018-03-03'), Row(date=u'2014-10-25'), Row(date=u'2016-02-06'), Row(date=u'2017-02-18'), Row(date=u'2015-09-19'), Row(date=u'2015-04-11'), Row(date=u'2013-04-13'), Row(date=u'2016-11-26'), Row(date=u'2015-04-04'), Row(date=u'2016-12-24'), Row(date=u'2013-12-07'), Row(date=u'2016-11-12'), Row(date=u'2015-10-24'), Row(date=u'2015-05-09'), Row(date=u'2017-03-04'), Row(date=u'2018-12-29'), Row(date=u'2019-03-23'), Row(date=u'2014-12-20'), Row(date=u'2016-03-12'), Row(date=u'2017-02-25'), Row(date=u'2019-03-30'), Row(date=u'2014-01-18'), Row(date=u'2013-07-20'), Row(date=u'2019-06-29'), Row(date=u'2017-03-25'), Row(date=u'2018-07-14'), Row(date=u'2016-03-05'), Row(date=u'2017-05-13'), Row(date=u'2013-02-02'), Row(date=u'2014-08-09'), Row(date=u'2019-05-04'), Row(date=u'2013-06-08'), Row(date=u'2019-02-23'), Row(date=u'2015-07-25'), Row(date=u'2016-08-06'), Row(date=u'2018-01-06'), Row(date=u'2016-05-28'), Row(date=u'2017-11-25'), Row(date=u'2018-05-19'), Row(date=u'2014-10-11'), Row(date=u'2014-03-22'), Row(date=u'2013-06-22'), Row(date=u'2014-08-30'), Row(date=u'2016-01-09'), Row(date=u'2013-01-12'), Row(date=u'2014-08-23'), Row(date=u'2015-08-29'), Row(date=u'2019-03-16'), Row(date=u'2015-08-01'), Row(date=u'2016-08-13'), Row(date=u'2018-11-10'), Row(date=u'2015-01-31'), Row(date=u'2013-11-09'), Row(date=u'2014-05-17'), Row(date=u'2017-09-16'), Row(date=u'2013-07-13'), Row(date=u'2014-03-01'), Row(date=u'2018-10-06'), Row(date=u'2017-11-04'), Row(date=u'2018-11-24'), Row(date=u'2013-09-14'), Row(date=u'2013-10-19'), Row(date=u'2013-05-18'), Row(date=u'2016-09-17'), Row(date=u'2017-04-29'), Row(date=u'2014-01-25'), Row(date=u'2018-02-03'), Row(date=u'2017-05-27'), Row(date=u'2013-03-09'), Row(date=u'2019-10-19'), Row(date=u'2019-05-25'), Row(date=u'2013-02-23'), Row(date=u'2018-02-10'), Row(date=u'2015-03-21'), Row(date=u'2017-10-21'), Row(date=u'2014-06-07'), Row(date=u'2013-09-07'), Row(date=u'2019-04-20'), Row(date=u'2014-01-04'), Row(date=u'2019-08-17'), Row(date=u'2013-06-29'), Row(date=u'2016-07-09'), Row(date=u'2017-01-14'), Row(date=u'2019-08-10'), Row(date=u'2018-03-17'), Row(date=u'2017-04-22'), Row(date=u'2014-02-01'), Row(date=u'2017-09-09'), Row(date=u'2017-03-11'), Row(date=u'2014-11-29'), Row(date=u'2019-02-02'), Row(date=u'2019-05-11'), Row(date=u'2017-07-08'), Row(date=u'2014-06-21'), Row(date=u'2018-01-27'), Row(date=u'2015-12-26'), Row(date=u'2013-09-21'), Row(date=u'2015-10-17'), Row(date=u'2013-05-04'), Row(date=u'2015-12-05'), Row(date=u'2014-04-26'), Row(date=u'2018-03-31'), Row(date=u'2017-04-08'), Row(date=u'2015-10-31'), Row(date=u'2013-08-17'), Row(date=u'2019-07-06'), Row(date=u'2017-11-18'), Row(date=u'2019-11-02'), Row(date=u'2013-05-25'), Row(date=u'2019-09-14'), Row(date=u'2013-04-20'), Row(date=u'2013-03-30'), Row(date=u'2013-11-02'), Row(date=u'2015-11-07'), Row(date=u'2017-08-05'), Row(date=u'2013-05-11'), Row(date=u'2015-02-21'), Row(date=u'2013-03-02'), Row(date=u'2017-01-28'), Row(date=u'2013-08-10'), Row(date=u'2015-08-08'), Row(date=u'2017-01-21'), Row(date=u'2018-06-16'), Row(date=u'2014-12-06'), Row(date=u'2017-07-22'), Row(date=u'2017-07-15'), Row(date=u'2014-07-05'), Row(date=u'2016-02-27'), Row(date=u'2018-01-13'), Row(date=u'2015-09-26'), Row(date=u'2018-03-24'), Row(date=u'2014-05-31'), Row(date=u'2018-09-22'), Row(date=u'2016-09-24'), Row(date=u'2019-07-27'), Row(date=u'2013-03-16'), Row(date=u'2017-10-07'), Row(date=u'2015-06-27'), Row(date=u'2014-11-15'), Row(date=u'2013-04-27'), Row(date=u'2016-02-20'), Row(date=u'2013-06-01'), Row(date=u'2019-03-09'), Row(date=u'2015-12-19'), Row(date=u'2016-07-02'), Row(date=u'2016-12-03'), Row(date=u'2017-12-16'), Row(date=u'2017-05-06'), Row(date=u'2019-11-23'), Row(date=u'2016-02-13'), Row(date=u'2018-06-02'), Row(date=u'2017-01-07'), Row(date=u'2016-12-10'), Row(date=u'2017-12-23'), Row(date=u'2015-07-04'), Row(date=u'2016-04-16'), Row(date=u'2014-06-14'), Row(date=u'2017-09-30'), Row(date=u'2013-11-23'), Row(date=u'2017-09-02'), Row(date=u'2016-03-19'), Row(date=u'2014-04-19'), Row(date=u'2015-03-14'), Row(date=u'2017-12-30'), Row(date=u'2015-01-17'), Row(date=u'2019-09-28'), Row(date=u'2016-10-22'), Row(date=u'2014-11-22'), Row(date=u'2014-08-02'), Row(date=u'2017-08-19'), Row(date=u'2014-09-06'), Row(date=u'2019-04-27'), Row(date=u'2015-06-13'), Row(date=u'2015-08-22'), Row(date=u'2018-10-27'), Row(date=u'2018-09-08'), Row(date=u'2015-10-03'), Row(date=u'2019-07-20'), Row(date=u'2014-10-04'), Row(date=u'2018-08-11'), Row(date=u'2016-06-25'), Row(date=u'2018-11-03'), Row(date=u'2013-08-31'), Row(date=u'2018-10-20'), Row(date=u'2016-08-27'), Row(date=u'2019-08-24'), Row(date=u'2019-11-09'), Row(date=u'2017-06-17'), Row(date=u'2014-07-19'), Row(date=u'2014-03-29'), Row(date=u'2017-10-14'), Row(date=u'2016-04-02'), Row(date=u'2016-12-17'), Row(date=u'2017-08-12'), Row(date=u'2014-01-11'), Row(date=u'2018-02-24'), Row(date=u'2016-01-02'), Row(date=u'2018-05-05'), Row(date=u'2015-05-16'), Row(date=u'2019-01-19'), Row(date=u'2018-05-12'), Row(date=u'2017-11-11'), Row(date=u'2016-06-04'), Row(date=u'2019-02-16'), Row(date=u'2016-09-10'), Row(date=u'2013-11-30'), Row(date=u'2016-03-26'), Row(date=u'2019-06-01'), Row(date=u'2019-05-18'), Row(date=u'2017-07-01'), Row(date=u'2017-05-20'), Row(date=u'2013-12-21'), Row(date=u'2018-09-01'), Row(date=u'2015-09-05'), Row(date=u'2019-04-13'), Row(date=u'2017-04-01'), Row(date=u'2017-04-15'), Row(date=u'2013-01-26'), Row(date=u'2013-08-24'), Row(date=u'2014-09-13'), Row(date=u'2019-10-26'), Row(date=u'2016-11-05'), Row(date=u'2013-01-19'), Row(date=u'2019-03-02'), Row(date=u'2014-05-03'), Row(date=u'2014-03-15'), Row(date=u'2019-09-07'), Row(date=u'2015-04-25'), Row(date=u'2016-07-16'), Row(date=u'2016-04-30'), Row(date=u'2015-04-18'), Row(date=u'2019-02-09'), Row(date=u'2015-11-21'), Row(date=u'2015-09-12'), Row(date=u'2016-06-18'), Row(date=u'2019-08-03'), Row(date=u'2014-08-16'), Row(date=u'2015-01-03'), Row(date=u'2015-12-12'), Row(date=u'2014-12-27'), Row(date=u'2018-04-07'), Row(date=u'2018-05-26'), Row(date=u'2016-06-11'), Row(date=u'2013-02-16'), Row(date=u'2015-11-14'), Row(date=u'2016-04-23'), Row(date=u'2018-04-21'), Row(date=u'2016-10-29'), Row(date=u'2015-03-07'), Row(date=u'2018-03-10'), Row(date=u'2015-06-20'), Row(date=u'2018-08-25'), Row(date=u'2014-07-12'), Row(date=u'2014-03-08'), Row(date=u'2015-06-06'), Row(date=u'2013-10-12'), Row(date=u'2016-05-07'), Row(date=u'2017-06-10'), Row(date=u'2016-09-03'), Row(date=u'2018-12-08'), Row(date=u'2017-10-28'), Row(date=u'2015-05-23')]

    print collect_date
    date_list = [(x[0][:7],x[0]) for x in collect_date]

    return date_list
    
    
import traceback
def check_au_app_data_count(store_id_list, device_id_list, _granularity, date_list):
    t = unittest.TestCase('run')
    for id,country_code in store_id_list.items():
        for device,device_code in device_id_list.items():
            for m in date_list:
                raw_count_with_KPI=''
                # print id, device, m[0] , m[1]
                try:
                    raw_path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v1/fact/granularity={raw_granularity}/month={raw_month}/device_id={raw_device_id}/store_id={raw_store_id}/"
                    unified_path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.segmented-by-app-used.v1/fact/granularity={unified_granularity}/date={unified_date}/"
                    raw_path_parse=raw_path.format(raw_device_id=device,raw_store_id=id, raw_month=m[0], raw_granularity=_granularity)
                    raw_count_with_KPI=spark.read.parquet(raw_path_parse).filter("date={}".format(m[1])).select("kpi","app_id","seg_app_id").distinct().groupBy("kpi").agg(count("kpi")).collect()
                    # print raw_count_with_KPI
                except AnalysisException as e: 
                    pass
                    # print e
                    break
                    # traceback.print_exc()
                for row in raw_count_with_KPI:
                    # print 'row _ test', row["kpi"], row["count(kpi)"]
                    unified_path_parse=unified_path.format(unified_date=m[1], unified_granularity=_granularity)
                    unified_count= spark.read.parquet(unified_path_parse).filter("device_code='{}' and country_code='{}'".format(device_code,country_code)).filter("{} is not null".format(kpi_mapping[row["kpi"]])).select(kpi_mapping[row["kpi"]]).count()
                    # print 'unified count' , unified_count
                    t.assertEqual(row["count(kpi)"], unified_count, " raw: {} ~ unified data: {},device:{},  store_id:{} , month: {}, KPI {}".format(row["count(kpi)"], unified_count, device, id , m, row["kpi"]))
graularity_list=["weekly"]
for graularity in graularity_list:
    check_au_app_data_count(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),graularity, get_path_date_list(graularity))
print 'pass'



In [0]:

print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v1/fact/granularity=weekly/month=2014-02/device_id=1001/store_id=1/").filter("kpi=12 and date='2014-02-22'").count()

print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.segmented-by-app-used.v1/fact/granularity=weekly/date=2014-02-22").filter("device_code='android-phone' and country_code='AU' and est_share_of_category_time is not null").count()


In [0]:

print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v1/fact/granularity=weekly/month=2014-02/device_id=1001/store_id=1/").filter("kpi=12 and date='2014-02-22'").count()

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v1/fact/granularity=monthly/month=2014-02/device_id=1001/store_id=1/
 
 
 


In [0]:

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

from aaintdatapipeline.application.app_qa.db_check_v1.constants import pg_settings
from aaintdatapipeline.core.urn import Urn
from aaintdatapipeline.application.app_qa.conf import settings
from aaintdatapipeline.core.loader.plproxy import build_db_settings
from aaplproxy.connection import ClusterConnection
import unittest
import datetime
from dateutil.relativedelta import relativedelta
import psycopg2

ANDROID_COUNTRY_ID_CODES = { 1: 'AU', 2: 'CA', 3: 'CN', 4: 'DE', 5: 'ES', 6: 'FR', 7: 'GB', 8: 'IT', 9: 'JP', 10: 'US', 11: 'BE', 12: 'CH', 13: 'CL', 14: 'ZA', 15: 'VN', 16: 'HK', 17: 'AR', 18: 'BR', 19: 'IN', 20: 'FI', 21: 'ID', 22: 'RU', 23: 'NL', 24: 'MY', 25: 'TR', 26: 'MX', 27: 'KR', 28: 'PL', 29: 'TH', 30: 'TW', 31: 'PH', 32: 'SG', 33: 'EG', 34: 'SE', 35: 'AT', 36: 'CZ', 37: 'HU', 38: 'DK', 39: 'IE', 40: 'IL', 41: 'NZ', 42: 'NO', 43: 'PT', 44: 'RO', 45: 'SK', 46: 'GR', 47: 'BG', 48: 'UA', 49: 'AE', 50: 'KW', 51: 'SA', 52: 'CO', 65: 'LB', 56: 'PE', 80: 'HR', 54: 'PK', 62: 'EC', 73: 'QA', 102: 'MO', 103: 'LU', 53: 'KZ', 1000: 'WW' }


# IOS_COUNTRY_ID_CODES={143460:'AU',143455:'CA',143465:'CN',143443:'DE',143454:'ES'}

IOS_COUNTRY_ID_CODES={143460:'AU',143455:'CA',143465:'CN',143443:'DE',143454:'ES',143442:'FR',143444:'GB',143450:'IT',143462:'JP',143441:'US',143446:'BE',143459:'CH',143483:'CL',143472:'ZA',143471:'VN',143463:'HK',143505:'AR',143503:'BR',143467:'IN',143447:'FI',143476:'ID',143469:'RU',143452:'NL',143473:'MY',143480:'TR',143468:'MX',143466:'KR',143478:'PL',143475:'TH',143470:'TW',143474:'PH',143464:'SG',143516:'EG',143456:'SE',143445:'AT',143489:'CZ',143482:'HU',143458:'DK',143449:'IE',143491:'IL',143461:'NZ',143457:'NO',143453:'PT',143487:'RO',143496:'SK',143448:'GR',143526:'BG',143492:'UA',143481:'AE',143493:'KW',143479:'SA',143501:'CO',143451:'LU',143497:'LB',143515:'MO',143507:'PE',143494:'HR',143477:'PK',143509:'EC',143498:'QA',0:'WW'}

def get_device_list(device):
    if device=='ios':
        return ['2001','2002']
    else:
        return ['1001','1002']

def get_month_list():
    result = []
    today = datetime.date(2019, 10, 1) 
    current = datetime.date(2013, 1, 1)    
    while current <= today:
        month_data_raw=datetime.datetime.strftime(current,'%Y-%m')
        month_data_leg_db=datetime.datetime.strftime(current,'%Y%m')
        result.append((month_data_raw, month_data_leg_db ))
        current += relativedelta(months=1)    
    return result
    
    
def pg_settings(urn, sql, granularity, schema):
    result = {}
    db_settings = build_db_settings(urn, schema)
    connection = ClusterConnection(db_settings)
    master_runner = connection.master_runner
    rows, _ = master_runner.select(sql)
    result[granularity] = rows
    return result


urn = Urn(
    namespace='app-qa.db-validation.v1',
    owner='app_qa'
)

path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v1/fact/granularity={raw_granularity}/month={raw_month}/device_id={device}/store_id={sid}/"


import traceback
def check_au_app_dump_data(store_id_list, device_id_list, _granularity):
    t = unittest.TestCase('run')
    for id in store_id_list:
        for device in device_id_list:
            for m in get_month_list():
                # print id, device, m[0] 
                try:
                    weekly_basic = """
                        SELECT app_id_count FROM plproxy.execute_select_nestloop($$select count(app_id) from au.app_{granularity}_{device_id}_{store_id}_{month} $$) t (app_id_count BIGINT)
                    """.format(device_id=device,store_id=id, month=m[1], granularity=_granularity)
                    granularity_ = _granularity.capitalize()
                    result = pg_settings(urn, weekly_basic, granularity_,'usage')
                    l = [int(x[0]) for x in result[granularity_]]
                    if sum(l)==0:
                        pass
                        # print 'the table is empty  - device:{}  store_id:{} month:{}'.format(device, id, m[0])
                    else:
                        dump_data_path=path.format(device=device, raw_month=m[0], sid=id, raw_granularity=_granularity)
                        dump_count=spark.read.parquet(dump_data_path).count()
                        t.assertEqual(dump_count, sum(l), "{}:{} ~ raw: {} ~ dumped data: {},device:{},  store_id:{} , month: {}".format(granularity_, m[0], sum(l), dump_count, device, id , m))
                except psycopg2.ProgrammingError : 
                    pass
                    # traceback.print_exc()
                    # print(e)
                    #  print 'the table is not existed - granularity {}, device:{}  store_id:{} month:{}'.format(granularity_, device, id, m[0])
                

# check_au_app_dump_data(IOS_COUNTRY_ID_CODES, get_device_list('ios'),'monthly')
# check_au_app_dump_data(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),'monthly')
# check_au_app_dump_data(IOS_COUNTRY_ID_CODES, get_device_list('ios'),'weekly')
check_au_app_dump_data(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),'weekly')
print 'pass'



In [0]:

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

from aaintdatapipeline.application.app_qa.db_check_v1.constants import pg_settings
from aaintdatapipeline.core.urn import Urn
from aaintdatapipeline.application.app_qa.conf import settings
from aaintdatapipeline.core.loader.plproxy import build_db_settings
from aaplproxy.connection import ClusterConnection
import unittest
import datetime
from dateutil.relativedelta import relativedelta
import psycopg2

ANDROID_COUNTRY_ID_CODES = { 1: 'AU', 2: 'CA', 3: 'CN', 4: 'DE', 5: 'ES', 6: 'FR', 7: 'GB', 8: 'IT', 9: 'JP', 10: 'US', 11: 'BE', 12: 'CH', 13: 'CL', 14: 'ZA', 15: 'VN', 16: 'HK', 17: 'AR', 18: 'BR', 19: 'IN', 20: 'FI', 21: 'ID', 22: 'RU', 23: 'NL', 24: 'MY', 25: 'TR', 26: 'MX', 27: 'KR', 28: 'PL', 29: 'TH', 30: 'TW', 31: 'PH', 32: 'SG', 33: 'EG', 34: 'SE', 35: 'AT', 36: 'CZ', 37: 'HU', 38: 'DK', 39: 'IE', 40: 'IL', 41: 'NZ', 42: 'NO', 43: 'PT', 44: 'RO', 45: 'SK', 46: 'GR', 47: 'BG', 48: 'UA', 49: 'AE', 50: 'KW', 51: 'SA', 52: 'CO', 65: 'LB', 56: 'PE', 80: 'HR', 54: 'PK', 62: 'EC', 73: 'QA', 102: 'MO', 103: 'LU', 53: 'KZ', 1000: 'WW' }


IOS_COUNTRY_ID_CODES={143460:'AU',143455:'CA',143465:'CN',143443:'DE',143454:'ES',143442:'FR',143444:'GB',143450:'IT',143462:'JP',143441:'US',143446:'BE',143459:'CH',143483:'CL',143472:'ZA',143471:'VN',143463:'HK',143505:'AR',143503:'BR',143467:'IN',143447:'FI',143476:'ID',143469:'RU',143452:'NL',143473:'MY',143480:'TR',143468:'MX',143466:'KR',143478:'PL',143475:'TH',143470:'TW',143474:'PH',143464:'SG',143516:'EG',143456:'SE',143445:'AT',143489:'CZ',143482:'HU',143458:'DK',143449:'IE',143491:'IL',143461:'NZ',143457:'NO',143453:'PT',143487:'RO',143496:'SK',143448:'GR',143526:'BG',143492:'UA',143481:'AE',143493:'KW',143479:'SA',143501:'CO',143451:'LU',143497:'LB',143515:'MO',143507:'PE',143494:'HR',143477:'PK',143509:'EC',143498:'QA',0:'WW'}

def get_device_list(device):
    if device=='ios':
        return ['2001','2002']
    else:
        return ['1001','1002']
   # return ['1001','1002','2001','2002']

def get_month_list():
    result = []
    today = datetime.date(2019, 10, 1) 
    current = datetime.date(2013, 1, 1)    
    while current <= today:
        month_data_raw=datetime.datetime.strftime(current,'%Y-%m')
        month_data_leg_db=datetime.datetime.strftime(current,'%Y%m')
        result.append((month_data_raw, month_data_leg_db ))
        current += relativedelta(months=1)    
    return result
    
    
def pg_settings(urn, sql, granularity, schema):
    result = {}
    db_settings = build_db_settings(urn, schema)
    connection = ClusterConnection(db_settings)
    master_runner = connection.master_runner
    rows, _ = master_runner.select(sql)
    result[granularity] = rows
    return result


urn = Urn(
    namespace='app-qa.db-validation.v1',
    owner='app_qa'
)

path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v1/fact/granularity=monthly/month={raw_month}/device_id={device}/store_id={sid}/"


import traceback
def check_mobile_web_basic_date(store_id_list, device_id_list):
    t = unittest.TestCase('run')
    for id in store_id_list:
        for device in device_id_list:
            for m in get_month_list():
                print id, device, m[0] 
                try:
                    weekly_basic = """
                        SELECT app_id_count FROM plproxy.execute_select_nestloop($$select count(app_id) from mu.app_monthly_{device_id}_{store_id}_{month} $$) t (app_id_count BIGINT)
                    """.format(device_id=device,store_id=id, month=m[1])
                    granularity_ = 'Monthly'
                    result = pg_settings(urn, weekly_basic, granularity_,'usage')
                    l = [int(x[0]) for x in result["Monthly"]]
                    if sum(l)==0:
                        print 'the table is empty  - device:{}  store_id:{} month:{}'.format(device, id, m[0])
                    else:
                        dump_data_path=path.format(device=device, raw_month=m[0], sid=id)
                        dump_count=spark.read.parquet(dump_data_path).count()
                        t.assertEqual(dump_count, sum(l), "month:{} ~ raw: {} ~ dumped data: {} ".format(m[0], dump_count, sum(l)))
                except psycopg2.ProgrammingError : 
                    # traceback.print_exc()
                    # print(e)
                    print 'the table is not existed - device:{}  store_id:{} month:{}'.format(device, id, m[0])
                


check_mobile_web_basic_date(IOS_COUNTRY_ID_CODES, get_device_list('ios'))
#check_mobile_web_basic_date(ANDROID_COUNTRY_ID_CODES, get_device_list('android'))

#  > /tmp/count_mu.txt

In [0]:

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

from aaintdatapipeline.application.app_qa.db_check_v1.constants import pg_settings
from aaintdatapipeline.core.urn import Urn
from aaintdatapipeline.application.app_qa.conf import settings
from aaintdatapipeline.core.loader.plproxy import build_db_settings
from aaplproxy.connection import ClusterConnection
import unittest
import datetime
from dateutil.relativedelta import relativedelta
import psycopg2

ANDROID_COUNTRY_ID_CODES = { 1: 'AU', 2: 'CA', 3: 'CN', 4: 'DE', 5: 'ES', 6: 'FR', 7: 'GB', 8: 'IT', 9: 'JP', 10: 'US', 11: 'BE', 12: 'CH', 13: 'CL', 14: 'ZA', 15: 'VN', 16: 'HK', 17: 'AR', 18: 'BR', 19: 'IN', 20: 'FI', 21: 'ID', 22: 'RU', 23: 'NL', 24: 'MY', 25: 'TR', 26: 'MX', 27: 'KR', 28: 'PL', 29: 'TH', 30: 'TW', 31: 'PH', 32: 'SG', 33: 'EG', 34: 'SE', 35: 'AT', 36: 'CZ', 37: 'HU', 38: 'DK', 39: 'IE', 40: 'IL', 41: 'NZ', 42: 'NO', 43: 'PT', 44: 'RO', 45: 'SK', 46: 'GR', 47: 'BG', 48: 'UA', 49: 'AE', 50: 'KW', 51: 'SA', 52: 'CO', 65: 'LB', 56: 'PE', 80: 'HR', 54: 'PK', 62: 'EC', 73: 'QA', 102: 'MO', 103: 'LU', 53: 'KZ', 1000: 'WW' }


IOS_COUNTRY_ID_CODES={143460:'AU',143455:'CA',143465:'CN',143443:'DE',143454:'ES',143442:'FR',143444:'GB',143450:'IT',143462:'JP',143441:'US',143446:'BE',143459:'CH',143483:'CL',143472:'ZA',143471:'VN',143463:'HK',143505:'AR',143503:'BR',143467:'IN',143447:'FI',143476:'ID',143469:'RU',143452:'NL',143473:'MY',143480:'TR',143468:'MX',143466:'KR',143478:'PL',143475:'TH',143470:'TW',143474:'PH',143464:'SG',143516:'EG',143456:'SE',143445:'AT',143489:'CZ',143482:'HU',143458:'DK',143449:'IE',143491:'IL',143461:'NZ',143457:'NO',143453:'PT',143487:'RO',143496:'SK',143448:'GR',143526:'BG',143492:'UA',143481:'AE',143493:'KW',143479:'SA',143501:'CO',143451:'LU',143497:'LB',143515:'MO',143507:'PE',143494:'HR',143477:'PK',143509:'EC',143498:'QA',0:'WW'}

def get_device_list(device):
    if device=='ios':
        return ['2001','2002']
    else:
        return ['1001','1002']
   # return ['1001','1002','2001','2002']

def get_month_list():
    result = []
    today = datetime.date(2019, 10, 1) 
    current = datetime.date(2013, 1, 1)    
    while current <= today:
        month_data_raw=datetime.datetime.strftime(current,'%Y-%m')
        month_data_leg_db=datetime.datetime.strftime(current,'%Y%m')
        result.append((month_data_raw, month_data_leg_db ))
        current += relativedelta(months=1)    
    return result
    
    
def pg_settings(urn, sql, granularity, schema):
    result = {}
    db_settings = build_db_settings(urn, schema)
    connection = ClusterConnection(db_settings)
    master_runner = connection.master_runner
    rows, _ = master_runner.select(sql)
    result[granularity] = rows
    return result


urn = Urn(
    namespace='app-qa.db-validation.v1',
    owner='app_qa'
)

path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v1/fact/granularity={raw_granularity}/month={raw_month}/device_id={device}/store_id={sid}/"


import traceback
def check_mobile_web_basic_date(store_id_list, device_id_list, _granularity):
    t = unittest.TestCase('run')
    for id in store_id_list:
        for device in device_id_list:
            for m in get_month_list():
                # print id, device, m[0] 
                try:
                    weekly_basic = """
                        SELECT app_id_count FROM plproxy.execute_select_nestloop($$select count(app_id) from mu.app_{granularity}_{device_id}_{store_id}_{month} $$) t (app_id_count BIGINT)
                    """.format(device_id=device,store_id=id, month=m[1], granularity=_granularity)
                    granularity_ = _granularity.capitalize()
                    result = pg_settings(urn, weekly_basic, granularity_,'usage')
                    l = [int(x[0]) for x in result[granularity_]]
                    if sum(l)==0:
                        pass
                        # print 'the table is empty  - device:{}  store_id:{} month:{}'.format(device, id, m[0])
                    else:
                        dump_data_path=path.format(device=device, raw_month=m[0], sid=id, raw_granularity=_granularity)
                        dump_count=spark.read.parquet(dump_data_path).count()
                        t.assertEqual(dump_count, sum(l), "{}:{} ~ raw: {} ~ dumped data: {} ".format(granularity_, m[0], dump_count, sum(l)))
                except psycopg2.ProgrammingError : 
                    pass
                    # traceback.print_exc()
                    # print(e)
                    # print 'the table is not existed - granularity {}, device:{}  store_id:{} month:{}'.format(granularity_, device, id, m[0])
                


check_mobile_web_basic_date(IOS_COUNTRY_ID_CODES, get_device_list('ios'),'weekly')
check_mobile_web_basic_date(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),'weekly')
check_mobile_web_basic_date(IOS_COUNTRY_ID_CODES, get_device_list('ios'),'daily')
check_mobile_web_basic_date(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),'daily')
print 'pass'

#  > /tmp/count_mu.txt

In [0]:

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

from aaintdatapipeline.application.app_qa.db_check_v1.constants import pg_settings
from aaintdatapipeline.core.urn import Urn
from aaintdatapipeline.application.app_qa.conf import settings
from aaintdatapipeline.core.loader.plproxy import build_db_settings
from aaplproxy.connection import ClusterConnection
import unittest
import datetime
from dateutil.relativedelta import relativedelta
import psycopg2

ANDROID_COUNTRY_ID_CODES = { 1: 'AU', 2: 'CA', 3: 'CN', 4: 'DE', 5: 'ES', 6: 'FR', 7: 'GB', 8: 'IT', 9: 'JP', 10: 'US', 11: 'BE', 12: 'CH', 13: 'CL', 14: 'ZA', 15: 'VN', 16: 'HK', 17: 'AR', 18: 'BR', 19: 'IN', 20: 'FI', 21: 'ID', 22: 'RU', 23: 'NL', 24: 'MY', 25: 'TR', 26: 'MX', 27: 'KR', 28: 'PL', 29: 'TH', 30: 'TW', 31: 'PH', 32: 'SG', 33: 'EG', 34: 'SE', 35: 'AT', 36: 'CZ', 37: 'HU', 38: 'DK', 39: 'IE', 40: 'IL', 41: 'NZ', 42: 'NO', 43: 'PT', 44: 'RO', 45: 'SK', 46: 'GR', 47: 'BG', 48: 'UA', 49: 'AE', 50: 'KW', 51: 'SA', 52: 'CO', 65: 'LB', 56: 'PE', 80: 'HR', 54: 'PK', 62: 'EC', 73: 'QA', 102: 'MO', 103: 'LU', 53: 'KZ', 1000: 'WW' }


IOS_COUNTRY_ID_CODES={143460:'AU',143455:'CA',143465:'CN',143443:'DE',143454:'ES',143442:'FR',143444:'GB',143450:'IT',143462:'JP',143441:'US',143446:'BE',143459:'CH',143483:'CL',143472:'ZA',143471:'VN',143463:'HK',143505:'AR',143503:'BR',143467:'IN',143447:'FI',143476:'ID',143469:'RU',143452:'NL',143473:'MY',143480:'TR',143468:'MX',143466:'KR',143478:'PL',143475:'TH',143470:'TW',143474:'PH',143464:'SG',143516:'EG',143456:'SE',143445:'AT',143489:'CZ',143482:'HU',143458:'DK',143449:'IE',143491:'IL',143461:'NZ',143457:'NO',143453:'PT',143487:'RO',143496:'SK',143448:'GR',143526:'BG',143492:'UA',143481:'AE',143493:'KW',143479:'SA',143501:'CO',143451:'LU',143497:'LB',143515:'MO',143507:'PE',143494:'HR',143477:'PK',143509:'EC',143498:'QA',0:'WW'}

def get_device_list(device):
    if device=='ios':
        return ['2001','2002']
    else:
        return ['1001','1002']

def get_month_list():
    result = []
    today = datetime.date(2019, 10, 1) 
    current = datetime.date(2013, 1, 1)    
    while current <= today:
        month_data_raw=datetime.datetime.strftime(current,'%Y-%m')
        month_data_leg_db=datetime.datetime.strftime(current,'%Y%m')
        result.append((month_data_raw, month_data_leg_db ))
        current += relativedelta(months=1)    
    return result
    
    
def pg_settings(urn, sql, granularity, schema):
    result = {}
    db_settings = build_db_settings(urn, schema)
    connection = ClusterConnection(db_settings)
    master_runner = connection.master_runner
    rows, _ = master_runner.select(sql)
    result[granularity] = rows
    return result


urn = Urn(
    namespace='app-qa.db-validation.v1',
    owner='app_qa'
)

path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_category.v1/fact/granularity={raw_granularity}/month={raw_month}/device_id={device}/store_id={sid}/"


import traceback
def check_mu_category_dump_data(store_id_list, device_id_list, _granularity):
    t = unittest.TestCase('run')
    for id in store_id_list:
        for device in device_id_list:
            for m in get_month_list():
                # print id, device, m[0] 
                try:
                    weekly_basic = """
                        SELECT app_id_count FROM plproxy.execute_select_nestloop($$select count(app_id) from mu.category_{granularity}_{device_id}_{store_id}_{month} $$) t (app_id_count BIGINT)
                    """.format(device_id=device,store_id=id, month=m[1], granularity=_granularity)
                    granularity_ = _granularity.capitalize()
                    result = pg_settings(urn, weekly_basic, granularity_,'usage')
                    l = [int(x[0]) for x in result[granularity_]]
                    if sum(l)==0:
                        pass
                        # print 'the table is empty  - device:{}  store_id:{} month:{}'.format(device, id, m[0])
                    else:
                        dump_data_path=path.format(device=device, raw_month=m[0], sid=id, raw_granularity=_granularity)
                        dump_count=spark.read.parquet(dump_data_path).count()
                        t.assertEqual(dump_count, sum(l), "{}:{} ~ raw: {} ~ dumped data: {} ".format(granularity_, m[0], dump_count, sum(l)))
                except psycopg2.ProgrammingError : 
                    pass
                    # traceback.print_exc()
                    # print(e)
                    # print 'the table is not existed - granularity {}, device:{}  store_id:{} month:{}'.format(granularity_, device, id, m[0])
                

check_mu_category_dump_data(IOS_COUNTRY_ID_CODES, get_device_list('ios'),'monthly')
check_mu_category_dump_data(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),'monthly')
check_mu_category_dump_data(IOS_COUNTRY_ID_CODES, get_device_list('ios'),'weekly')
check_mu_category_dump_data(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),'weekly')
check_mu_category_dump_data(IOS_COUNTRY_ID_CODES, get_device_list('ios'),'daily')
check_mu_category_dump_data(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),'daily')
print 'pass'



In [0]:

from dateutil.relativedelta import relativedelta
import datetime
# date1='2019-01-01'
# date2='2019-01-01'
# date_list=[]
# start = datetime.datetime.strptime(date1, '%Y-%m-%d')
# end = datetime.datetime.strptime(date2, '%Y-%m-%d')
# step = datetime.timedelta(month=1)
# while start <= end:
#         date_list.append(str(start.date()))
#         # print start.date()
#         start += step
# print date_list
# print "','".join(date_list)


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)  
    return next_month - datetime.timedelta(days=next_month.day)


def get_month_list():
    result = []
    today = datetime.date(2019, 10, 1) 
    current = datetime.date(2013, 1, 1)    
    while current <= today:
        month_data_raw=datetime.datetime.strftime(current,'%Y-%m')
        month_data_leg_db=datetime.datetime.strftime(current,'%Y%m')
        result.append((month_data_raw, month_data_leg_db,current  ))
        current += relativedelta(months=1)  
    return result


# result = []
# today = datetime.date(2019, 11, 1) 
# current = datetime.date(2013, 1, 1)    

# while current <= today:
#     month_data_raw=datetime.datetime.strftime(current,'%Y-%m')
#     month_data_leg_db=datetime.datetime.strftime(current,'%Y%m')
#     # print month_data_raw , month_data_leg_db
#     result.append((month_data_raw, month_data_leg_db ))
#     current += relativedelta(months=1)

# print result
# for x in result:
#     print x[0],x[1]
    
print get_month_list()
for x in get_month_list():
    print last_day_of_month(x[2])


# IOS_COUNTRY_ID_CODES={143460:'AU',143455:'CA',143465:'CN',143443:'DE',143454:'ES',143442:'FR',143444:'GB',143450:'IT',143462:'JP',143441:'US',143446:'BE',143459:'CH',143483:'CL',143472:'ZA',143471:'VN',143463:'HK',143505:'AR',143503:'BR',143467:'IN',143447:'FI',143476:'ID',143469:'RU',143452:'NL',143473:'MY',143480:'TR',143468:'MX',143466:'KR',143478:'PL',143475:'TH',143470:'TW',143474:'PH',143464:'SG',143516:'EG',143456:'SE',143445:'AT',143489:'CZ',143482:'HU',143458:'DK',143449:'IE',143491:'IL',143461:'NZ',143457:'NO',143453:'PT',143487:'RO',143496:'SK',143448:'GR',143526:'BG',143492:'UA',143481:'AE',143493:'KW',143479:'SA',143501:'CO',143451:'LU',143497:'LB',143515:'MO',143507:'PE',143494:'HR',143477:'PK',143509:'EC',143498:'QA',0:'WW'}

# for x in IOS_COUNTRY_ID_CODES:
#     print x

In [0]:
%python
from itertools import islice

count=0
with open('/tmp/count_mu.txt') as f:
    for line in islice(f, 2, 10):
        count=count+int(line)
print count


In [0]:
%%sh
re='[0-9]+$'

count=0
while read p; do
  echo "$p"
  if ! [[ $p = $re ]] ; then
    echo $p
    count=`expr $count + $p`
  fi
done </tmp/count_mu.txt

echo $count


In [0]:

kpi_mapping={1:"est_average_active_users", 2: "est_average_session_per_user", 3:"est_average_session_duration", 4:"est_install_penetration", 5:"est_average_active_days", 6:"est_percentage_active_days", 7:"est_average_bytes_per_user" , 8:"est_average_time_per_user", 9:"est_usage_penetration", 10:"est_open_rate", 12:"est_share_of_category_time", 15:"est_share_of_category_session", 17:"est_average_bytes_per_session", 18:"est_share_of_category_bytes", 20:"est_percent_of_wifi_total", 21:"est_mb_per_second", 22:"est_panel_size", 23:"est_installs", 24:"est_average_active_users_country_share", 25:"est_installs_country_share"}

kpi_mapping[1]

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v1/fact/granularity=monthly/month=2013-01/

 

 

In [0]:

from pyspark.sql import functions as F
from pyspark.storagelevel import StorageLevel
KPI_LIST = ['est_average_active_users', 'est_average_session_per_user', 'est_average_session_duration',
            'est_install_penetration', 'est_average_active_days', 'est_percentage_active_days',
            'est_average_bytes_per_user', 'est_average_time_per_user', 'est_usage_penetration', 'est_open_rate',
            'est_share_of_category_time', 'est_share_of_category_session', 'est_average_bytes_per_session',
            'est_share_of_category_bytes', 'est_percent_of_wifi_total', 'est_mb_per_second', 'est_installs',
            'est_average_active_users_country_share', 'est_installs_country_share']
EXCLUDE_KPI_ID = [11, 13, 14, 16, 19, 22, 26, 27, 28]
base_path_dump = "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v1/fact/granularity={granularity}/month={month_str}/"
base_path_transformed = "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.basic-kpi.v1/fact/granularity={granularity}/date={date}/"
GRANULARIY_LIST = ["daily"]
def check_ouput(granularity, date):
    month_str = date[0:7]
    path_dump = base_path_dump.format(**{'month_str': month_str, 'granularity': granularity})
    path_transformed = base_path_transformed.format(**{'date': date, 'granularity': granularity})
    df_dump = spark.read.parquet(path_dump)
    df_dump = (
        df_dump
            .where("date='{}'".format(date))
    )
    dump_count = df_dump.select("app_id").dropDuplicates().count()
    df_dump = (
        df_dump
            .filter(~df_dump.kpi.isin(EXCLUDE_KPI_ID))
            .groupBy("app_id")
            .agg(F.col("app_id"), F.sum("estimate"))
            .select('app_id', 'sum(estimate)')
            .withColumnRenamed('sum(estimate)', 'raw_sum_est')
    )
    df_transformed = spark.read.parquet(path_transformed)
    df_transformed = df_transformed.na.fill(0)
    df_transformed = df_transformed.withColumn('sum_kpi', sum([df_transformed[col] for col in KPI_LIST]))
    df_transformed = (
        df_transformed
            .groupBy("app_id")
            .agg(F.col("app_id"), F.sum("sum_kpi"))
            .select('app_id', 'sum(sum_kpi)')
            .withColumnRenamed('sum(sum_kpi)', 'trans_sum_est')
            .withColumnRenamed('app_id', 'trans_app_id')
    )
    df_transformed.persist(StorageLevel.MEMORY_AND_DISK)
    transformed_count = df_transformed.count()
    df_join = (
        df_dump
            .join(df_transformed, df_dump['app_id'] == df_transformed['trans_app_id'], "inner")
    )
    df_join = (
        df_join
            .withColumn('minus', F.abs(df_join['trans_sum_est'] - df_join['raw_sum_est']))
            .where('minus > 0.001')
    )
    count_check_str = '{}_{}, dump_count:{},transformed_count:{},minus:{}'.format(granularity, date, dump_count,
                                                                                  transformed_count,
                                                                                  dump_count - transformed_count)
    write_to_file('/tmp/transform_qa.log', count_check_str)
    if df_join.count() > 0:
        df_join = (
            df_join
                .withColumn('granularity', F.lit(granularity))
                .withColumn('date', F.lit(date))
                .select('app_id', 'granularity', 'date', 'trans_sum_est', 'raw_sum_est', 'minus')
        )
        df_join.toPandas().to_csv('/tmp/transform_qa.csv'.format(), mode='a', header=False)
def write_to_file(filename, str):
    with open(filename, 'a') as file_object:
        file_object.write(str + "\n")
for granularity in GRANULARIY_LIST:
    df_date = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.basic-kpi.v1/fact/granularity={}/".format(granularity)).select('date').dropDuplicates()
    DATE_LIST =[str(row.date) for row in df_date.collect()]
    for date in DATE_LIST:
        check_ouput(granularity, date)
